In [1]:

import pickle
import math
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch.nn.functional as F
import numpy as np
import torch
import torch.nn as nn
from torch import optim
from torchvision import datasets, transforms, models
from torch.autograd import Variable
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler
import xlsxwriter


#data directory
data_dir = r'C:\Users\alexp\Desktop\MCF7_classification\viable_apop\traintest\Vtest'
#data_dir = r'C:\Users\alexp\Documents\MATLAB\Dr_Hu_Projects\Clustering\relabel\new_class\Vtest'
#data_dir = r'C:\Users\pinai\OneDrive - East Carolina University\Desktop\MCF7_classification\viable_apop\traintest\Vtest'
#data_dir = r'C:\Users\pinai\OneDrive - East Carolina University\Documents\MATLAB\Dr_Hu_Projects\Clustering\relabel\new_class\Vtest'

num_classes = 3
resize = 160
totalin = 5120

test_transforms = transforms.Compose([transforms.Resize(resize),
                                      transforms.
                                      transforms.ToTensor(),
                                      ]) 

test_data = datasets.ImageFolder(data_dir,
                    transform=test_transforms)


test_loader = torch.utils.data.DataLoader(test_data)


def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, 
                     stride=stride, padding=0, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

#DINet-R2 with feature extraction
z_stor = torch.zeros(len(test_data), totalin)
ft = 0
# ResNet
class ResNet(nn.Module):
	def __init__(self, block, layers, num_classes = num_classes):
		super(ResNet, self).__init__()
		self.in_channels = 16
		self.conv = conv3x3(3, 16)
		self.bn = nn.BatchNorm2d(16)
		self.relu = nn.ReLU(inplace=True)
		self.layer1 = self.make_layer(block, 32, layers[0])
		self.layer2 = self.make_layer(block, 64, layers[1])
		self.layer3 = self.make_layer(block, 128, layers[2])
		self.pool = nn.AvgPool2d(3, stride = 2)
#		self.pool2 = nn.AvgPool2d(2, stride = 1)
		self.conv4 = conv3x3(128, 256)
		self.bn4 = nn.BatchNorm2d(256)
		self.fc1 = nn.Linear(totalin, num_classes)
		self.softmax = nn.Softmax(dim=1)
       
	def make_layer(self, block, out_channels, blocks, stride=1):
		downsample = None
		if (stride != 1) or (self.in_channels != out_channels):
			downsample = nn.Sequential(
				conv1x1(self.in_channels, out_channels, stride=stride),
				nn.BatchNorm2d(out_channels))
		layers = []
		layers.append(block(self.in_channels, out_channels, stride, downsample))
		self.in_channels = out_channels
		for i in range(1, blocks):
			layers.append(block(out_channels, out_channels))
		return nn.Sequential(*layers)
			

	def forward(self, x):
		out = self.pool(x)
		#print(out.size())
		out = self.conv(out)
		out = self.bn(out)
		out = self.relu(out)
		out = self.pool(out)
		#print(out.size())
		out = self.layer1(out)
#		out = self.pool2(out)
		#print(out.size())
		out = self.layer2(out)
		#print(out.size())
		out = self.pool(out)
		#print(out.size())
		out = self.layer3(out)
		#print(out.size())
		out = self.pool(out)
		#print(out.size())
		out = self.conv4(out)
		out = self.bn4(out)
		out = self.relu(out)
		out = self.pool(out)
#		print(out.size())
		out = out.view(out.size(0), -1)

		#feature extraction
		imgg = out[0]
		for ft in range(totalin):
			fox = imgg[ft]
			z_stor[steps-1, ft] = z_stor[steps-1, ft] + fox
			#print(steps)

		out = self.fc1(out)
		out = self.softmax(out)
		return out


#torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = torch.load('ResNet.pth')
model

#testing the model and extracting labels
with torch.no_grad():
    conf_table = torch.zeros(num_classes, num_classes)
    correct = 0
    total = 0
    lbl_stor = torch.zeros(len(test_loader))
    name_stor = []
    steps = 0
    model.eval()
    for images, labels in test_loader:
        images, labels = images.cuda(), labels.cuda()
        steps += 1
        img_name = test_data.imgs[steps-1]
        outputs = model(images)
        _, prediction = torch.max(outputs.data, 1)
        lbl_stor[steps-1] = lbl_stor[steps-1] + prediction
        name_stor.append(img_name)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()
        accuracy = correct/total

        #confusion matrix with accuracy results
        for p, t in zip(prediction.view(-1), labels.view(-1)):
                conf_table[p.long(), t.long()] += 1
                confusion_matrix = torch.round(conf_table) 
                conf_sum = torch.sum(confusion_matrix)


print(confusion_matrix)
print("accuracy: {}".format(accuracy))
print("sum of elements: {}".format(conf_sum))


tensor([[161.,   6.,   6.],
        [  5., 165.,   0.],
        [  5.,   0., 165.]])
accuracy: 0.9571150097465887
sum of elements: 513.0


In [ ]:
######################################################################################################################

In [10]:

grpno = 5

#building a feature and label matrix for each cell class
znlbl = torch.zeros(len(test_loader), totalin + 1, num_classes)

for rw, ZZ in enumerate(z_stor):
	lbl = lbl_stor[rw]
	lbl = int(lbl.item())
	znlbl[rw,0,lbl] = znlbl[rw,0,lbl] + rw
	znlbl[rw,1:,lbl] = znlbl[rw,1:,lbl] + ZZ

znlbl = znlbl.detach().numpy()

znlbl0 = znlbl[:,:,0]
znlbl_0 = znlbl0[np.sum( np.absolute(znlbl0), 1) != 0]

znlbl1 = znlbl[:,:,1]
znlbl_1 = znlbl1[np.sum( np.absolute(znlbl1), 1) != 0]

znlbl2 = znlbl[:,:,2]
znlbl_2 = znlbl2[np.sum( np.absolute(znlbl2), 1) != 0]

zlabels = [znlbl_0,znlbl_1,znlbl_2]

#matching the path, class/label, and features to organize the images
lbl=-1
for xxx in zlabels:
    lbl=lbl+1

    claspath = []
    nosupaths = []
    znosupp = []
    zclass = []
    for igg, zzz in enumerate(xxx):
    
    	nsid = xxx[igg,0]
    	nosupimg = name_stor[int(nsid)]
    
    	if lbl_stor[int(nsid)] == lbl and name_stor[int(nsid)][1] == lbl:
    		claspath.append(nosupimg)
    		zclass.append(zzz[1:])
    
    	else:
    		nosupaths.append(nosupimg)
    		znosupp.append(zzz[1:])
    
    #building a dat file of the correctly classified cells
    suppbook = []
    for rrw, vls in enumerate(zclass):
            
        rpath = claspath[rrw][0]
        supclass = int(lbl)
        vlis = vls.tolist()
            
        ptclaf = [rpath, supclass] + vlis
        suppbook.append(ptclaf)
        newbook = np.reshape(suppbook, [len(suppbook), totalin+2])
        
    #formm = len(suppbook)*[(totalin+2)*['%s']]
    np.savetxt(r'C:\Users\alexp\Documents\MATLAB\Dr_Hu_Projects\Clustering\relabel\supp\resupp_%d_%d.dat' %(grpno, lbl), newbook, delimiter = ',', fmt='%s')
    #np.savetxt(r'C:\Users\pinai\OneDrive - East Carolina University\Documents\MATLAB\Dr_Hu_Projects\Clustering\relabel\supp\resupp_%d_%d.dat' %(grpno, via_apopnum), newbook, delimiter = ',', fmt='%s')
        
    #building an dat file with the incorrectly classified cells
    nosuppbook = [];
    for rrw, vls in enumerate(znosupp):
        
        rpath = nosupaths[rrw][0]
        orig_class = int(nosupaths[rrw][1])
        vlis = vls.tolist()
            
        ptclaf = [rpath, orig_class] + vlis
        nosuppbook.append(ptclaf)
        nobook = np.reshape(nosuppbook, [len(nosuppbook), totalin+2])
        
        #np.savetxt(r'C:\Users\pinai\OneDrive - East Carolina University\Documents\MATLAB\Dr_Hu_Projects\Clustering\relabel\nosupp\renosupp_%d_%d.dat' %(grpno, via_apopnum), nosuppbook, delimiter = ',', fmt='%s')
    np.savetxt(r'C:\Users\alexp\Documents\MATLAB\Dr_Hu_Projects\Clustering\relabel\nosupp\renosupp_%d_%d.dat' %(grpno, lbl), nosuppbook, delimiter = ',', fmt='%s')


